In [1]:
import pandas as pd
import numpy as np
import re

# Load data processing

In [2]:
# Read original hourly load
region = pd.read_csv('Processed Data/2022/Reduced_Area_Region.csv')
load = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Load.csv')

In [3]:
# reduce column names to Area Names
def extract_name(name):
    match = re.search(r'_(\w+)_', name)
    return match.group(1) if match else name

load.columns = [extract_name(name) for name in load.columns]

print(load.columns)

Index(['Index', 'AESO', 'AVA', 'AZPS', 'BANC', 'BCHA', 'BPAT', 'CFE', 'CHPD',
       'CIPB', 'CIPV', 'CISC', 'CISD', 'DOPD', 'EPE', 'GCPD', 'IID', 'IPFE',
       'IPMV', 'IPTV', 'LDWP', 'NEVP', 'NWMT', 'PACW', 'PAID', 'PAUT', 'PAWY',
       'PGE', 'PNM', 'PSCO', 'PSEI', 'SCL', 'SPPC', 'SRP', 'TEPC', 'TH_Malin',
       'TH_Mead', 'TH_PV', 'TIDC', 'TPWR', 'VEA', 'WACM', 'WALC', 'WAUW',
       'Unnamed: 44'],
      dtype='object')


In [4]:
# filter load data and keep 8784 hours (rows) and drop last empty column
load = load.loc[1:8784]
load = load.drop(load.columns[-1], axis=1)

# set column index as index
load = load.set_index(load.columns[0])

# load data to numeric, originally string
load = load.apply(pd.to_numeric)

In [5]:
# Create an empty DataFrame for the results
load_reduced = pd.DataFrame()

# Loop over the unique values in df_map
for value in region['Zone'].unique():
    # Get the columns that are mapped to this value
    cols = region.loc[region['Zone'] == value, 'Area Name']
    
    # Sum these columns in df and add the result to df_result
    load_reduced[value] = load[cols].sum(axis=1)

In [6]:
sorted_columns = sorted(load_reduced.columns, key=lambda x: int(x))
load_reduced = load_reduced[sorted_columns]


In [8]:
load_reduced.to_csv('Processed Data/2022/Load.csv')

# Thermal generator processing

In [9]:
genlist = pd.read_csv('Public Data/GeneratorList.csv',header=2, encoding='ISO-8859-1', usecols=['Name','SubType','Area Name','InitialDispatch(MW)','ServiceStatus','DevStatus'])
thermalgenlist = pd.read_csv('Public Data/Thermal_General_Info.csv',header=1, encoding='ISO-8859-1', usecols=['GeneratorName','MustRun','Fuel Name','MinimumDownTime(hr)','MinimumUpTime(hr)','RampUp Rate(MW/minute)','RampDn Rate(MW/minute)','Startup Cost Fixed($)','Startup Fuel Name','StartFuel(MMBTu)','VOM Cost','QuickStart'])
thermalgencurve = pd.read_csv('Public Data/Thermal_IOCurve_Info.csv',header=1, encoding='ISO-8859-1',usecols=['Generator Name','If Use Generic IO Curve','Generic IO Curve Name','IONumBlock','IOMaxCap(MW)','IOMinCap(MW)','MinInput(MMBTu)','IncCap2(MW)','IncHR2(MMBTu/MWh)','IncCap3(MW)','IncHR3(MMBTu/MWh)','IncCap4(MW)','IncHR4(MMBTu/MWh)','IncCap5(MW)','IncHR5(MMBTu/MWh)','IncCap6(MW)','IncHR6(MMBTu/MWh)'])
fuelcost = pd.read_csv('Public Data/FuelCost_Monthly.csv',header=1, encoding='ISO-8859-1')
thermalgencurve = thermalgencurve[~thermalgencurve['If Use Generic IO Curve']]


In [10]:
# attach zone info to genlist
genlist['Zone'] = genlist['Area Name'].map(region.set_index('Area Name')['Zone'])
genlist = genlist.dropna(subset=['Zone'])

In [11]:
# calculate average fuel cost and reduce the dataframe to two columns ['Fuel Name','Fuel Cost($/MMBTu)']
fuelcost['Fuel Cost($/MMBTu)']=fuelcost.iloc[:, 3:15].mean(axis=1)
fuelcost = fuelcost.drop(fuelcost.columns[1:15], axis=1)

In [12]:
thermalgen = pd.merge(thermalgencurve, genlist, left_on='Generator Name', right_on='Name', how='inner')
thermalgen = pd.merge(thermalgen, thermalgenlist, left_on='Generator Name', right_on='GeneratorName', how='inner')

In [13]:
thermalgen = pd.merge(thermalgen, fuelcost, left_on='Fuel Name', right_on='Fuel Name', how='inner')
thermalgen = pd.merge(thermalgen, fuelcost, left_on='Startup Fuel Name', right_on='Fuel Name', how='inner', suffixes=(None, '_Start'))

In [14]:
# cost calculation base on fuel prices
thermalgen['Start Up Cost($)'] = thermalgen['Startup Cost Fixed($)'] + thermalgen['StartFuel(MMBTu)'] * thermalgen['Fuel Cost($/MMBTu)_Start']
thermalgen['NoLoadCost($)'] = thermalgen['MinInput(MMBTu)'] * thermalgen['Fuel Cost($/MMBTu)']
thermalgen['IncCost2($/MW)'] = thermalgen['IncHR2(MMBTu/MWh)'] * thermalgen['Fuel Cost($/MMBTu)']
thermalgen['IncCost3($/MW)'] = thermalgen['IncHR3(MMBTu/MWh)'] * thermalgen['Fuel Cost($/MMBTu)']
thermalgen['IncCost4($/MW)'] = thermalgen['IncHR4(MMBTu/MWh)'] * thermalgen['Fuel Cost($/MMBTu)']
thermalgen['IncCost5($/MW)'] = thermalgen['IncHR5(MMBTu/MWh)'] * thermalgen['Fuel Cost($/MMBTu)']
thermalgen['IncCost6($/MW)'] = thermalgen['IncHR6(MMBTu/MWh)'] * thermalgen['Fuel Cost($/MMBTu)']

In [15]:
thermalgen = thermalgen[(thermalgen['DevStatus'] == 'Existing')&(thermalgen['ServiceStatus'] == '#TRUE#')]

In [16]:
thermalgen['MustRun'] = thermalgen['MustRun'].astype(int)

In [17]:
thermalgen.loc[(thermalgen['InitialDispatch(MW)'] == 0) & (thermalgen['MustRun'] == 1), 'InitialDispatch(MW)'] = thermalgen['IOMinCap(MW)']

In [18]:
thermalgen

,Generator Name,If Use Generic IO Curve,Generic IO Curve Name,IONumBlock,IOMaxCap(MW),IOMinCap(MW),MinInput(MMBTu),IncCap2(MW),IncHR2(MMBTu/MWh),IncCap3(MW),...,Fuel Cost($/MMBTu),Fuel Name_Start,Fuel Cost($/MMBTu)_Start,Start Up Cost($),NoLoadCost($),IncCost2($/MW),IncCost3($/MW),IncCost4($/MW),IncCost5($/MW),IncCost6($/MW)
0,Ox_Mountain7,False,NaN,2,1.9,0.57,6.9080,1.330,12.120,0.000,...,2.378763,Bio_Landfill_Gas,2.378763,11.155225,16.432495,28.830608,0.000000,0.000000,0.0,0.0
1,Ox_Mountain6,False,NaN,4,1.9,0.57,9.6890,0.380,11.819,0.570,...,2.378763,Bio_Landfill_Gas,2.378763,11.155225,23.047835,28.114600,29.548994,32.862611,0.0,0.0
2,Ox_Mountain5,False,NaN,4,1.9,0.57,9.6890,0.380,11.819,0.570,...,2.378763,Bio_Landfill_Gas,2.378763,11.155225,23.047835,28.114600,29.548994,32.862611,0.0,0.0
3,Ox_Mountain4,False,NaN,4,1.9,0.57,9.6890,0.380,11.819,0.570,...,2.378763,Bio_Landfill_Gas,2.378763,11.155225,23.047835,28.114600,29.548994,32.862611,0.0,0.0
4,Ox_Mountain3,False,NaN,4,1.9,0.57,9.6890,0.380,11.819,0.570,...,2.378763,Bio_Landfill_Gas,2.378763,11.155225,23.047835,28.114600,29.548994,32.862611,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301,Raft River Geothermal,False,NaN,2,12.0,6.00,59.8886,6.000,4.110,0.000,...,0.000000,Geothermal,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1302,Fresno Peaker,False,NaN,2,21.9,13.90,214.6600,8.000,5.597,0.000,...,5.335888,NG_Cal_PG&E LT,5.335888,694.364198,1145.401664,29.864964,0.000000,0.000000,0.0,0.0
1303,Ben French ICE 1-5,False,NaN,2,10.0,2.50,10.0000,7.500,11.000,0.000,...,24.566520,DefaultFuel,9.990000,109.990000,245.665200,270.231720,0.000000,0.000000,0.0,0.0
1307,Spindle Hill 1,False,NaN,4,160.0,81.00,983.4004,26.333,7.002,26.334,...,3.434265,NG_Colorado_Cheyenne,3.434265,9206.915907,3377.257821,24.046725,25.392957,26.735755,0.0,0.0


In [19]:
thermalgen.to_csv('Processed Data/2022/ThermalGen_Full_C.csv')

In [20]:
columns_to_drop = ['If Use Generic IO Curve', 'Generic IO Curve Name','IONumBlock', 'MinInput(MMBTu)',
                   'IncHR2(MMBTu/MWh)', 'IncHR3(MMBTu/MWh)', 'IncHR4(MMBTu/MWh)', 'IncHR5(MMBTu/MWh)','IncHR6(MMBTu/MWh)',
                  'Name', 'GeneratorName', 'Fuel Name', 'Startup Cost Fixed($)', 'Startup Fuel Name', 'StartFuel(MMBTu)',
                  'Fuel Cost($/MMBTu)', 'Fuel Name_Start', 'Fuel Cost($/MMBTu)_Start']
thermalgen = thermalgen.drop(columns=columns_to_drop)

In [21]:
mask = thermalgen['InitialDispatch(MW)'] != 0
thermalgen.loc[mask, 'InitialDispatch(MW)'] = thermalgen.loc[mask, 'InitialDispatch(MW)'].clip(lower=thermalgen.loc[mask, 'IOMinCap(MW)'], upper=thermalgen.loc[mask, 'IOMaxCap(MW)'])

In [22]:
thermalgen['Initial Status'] = (thermalgen['InitialDispatch(MW)'] != 0).astype(int)

In [23]:
thermalgen.to_csv('Processed Data/2022/ThermalGen_C.csv')

# Thermal generator map

In [24]:
thermalgenzone = pd.read_csv('Processed Data/2022/ThermalGen_C.csv', usecols=['Generator Name','Zone'])

In [25]:
thermalgenmap = pd.get_dummies(thermalgenzone["Zone"], dtype=int)
thermalgenmap.to_csv('Processed Data/2022/ThermalGenMap_C.csv')

# Wind  processing

In [26]:
genlist = pd.read_csv('Public Data/GeneratorList.csv',header=2, encoding='ISO-8859-1', usecols=['Name','SubType','Area Name','InitialDispatch(MW)','DevStatus'])
hourlygenlist = pd.read_csv('Public Data/HourlyResource_General.csv',header=2, encoding='ISO-8859-1')

In [27]:
# attach zone info to genlist
genlist['Zone'] = genlist['Area Name'].map(region.set_index('Area Name')['Zone'])
genlist = genlist.dropna(subset=['Zone'])
windlist = hourlygenlist[hourlygenlist['Type'] == 'Wind']
windlist = pd.merge(windlist, genlist, left_on='GeneratorName', right_on='Name', how='inner')

In [28]:
columns_to_drop = ['GeneratorKey','Type','CommitmentShapeID','CommitmentMultiplier','DispatchShapeID','DispatchShapeName',
                  'DispatchMultiplier','CommitmentVariation(%)','CommitmentPdf','DispatchVariation(%)','DispatchPdf',
                   'Spillage','IfEnableMultiBlock','ModelType','IfUseInvertLoadingRatio','InvertLoadingRatio','TargetCapacityFactor','Name']
windlist = windlist.drop(columns=columns_to_drop)

In [29]:
# windlist = windlist[windlist['Commission Date'] <= '#2024-01-01#']
windlist = windlist[windlist['DevStatus'] == 'Existing']

In [30]:
sum(windlist[(windlist['Zone']==1) | (windlist['Zone']==2)| (windlist['Zone']==3)]['Capacity(MW)'])

6485.12

In [31]:
windlist.to_csv('Processed Data/2022/Wind_C.csv')

# Wind  profile

In [32]:
windshapelist = pd.read_csv('Processed Data/2022/Wind_C.csv', usecols=['GeneratorName','CommitmentShapeName','Capacity(MW)'])
windcruve = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Wind.csv',header=0, skiprows=list(range(8786, 8796)))

C:\Users\wenmi\AppData\Local\Temp\ipykernel_10824\760433521.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  windcruve = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Wind.csv',header=0, skiprows=list(range(8786, 8796)))


In [33]:
windcruve = windcruve.drop(index=0)

In [34]:
windcruve.columns = [re.sub(r'\.dat:.*', '', col_name) for col_name in windcruve.columns]


In [35]:
windprofile = pd.DataFrame()
for index, row in windshapelist.iterrows():
    generator_name = row['GeneratorName']
    wind_profile_name = row['CommitmentShapeName']
    capacity = row['Capacity(MW)']
    wind_profile = windcruve[wind_profile_name]
    adjusted_profile = wind_profile * capacity
    windprofile[generator_name] = adjusted_profile

C:\Users\wenmi\AppData\Local\Temp\ipykernel_10824\1997807084.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  windprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_10824\1997807084.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  windprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_10824\1997807084.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  

In [36]:
windprofile.to_csv('Processed Data/2022/WindProfile_C.csv')

# Wind  map

In [37]:
windzone = pd.read_csv('Processed Data/2022/Wind_C.csv', usecols=['GeneratorName','Zone'])

In [38]:
windzone

,GeneratorName,Zone
0,70724_ADS30_Spring Canyon 2-3,5.0
1,ADS30_WT_WACM_1,5.0
2,Alta_VII_G_29591_EQ,2.0
3,Alta_X_G_29596_EQ,2.0
4,Alta_XI_G1_29592_EQ,2.0
...,...,...
287,WINTEC PALM,2.0
288,Wolverine Creek,5.0
289,Zephyr Park,2.0
290,znSTWP_GEN-1,5.0


In [40]:
windmap = pd.get_dummies(windzone["Zone"], dtype=int)
windmap.to_csv('Processed Data/2022/WindMap_C.csv')

In [41]:
windprofile = pd.read_csv('Processed Data/2022/WindProfile_C.csv').iloc[:,1:].values
windmap = pd.read_csv('Processed Data/2022/WindMap_C.csv').iloc[:,1:].values

windzone = pd.DataFrame(np.matmul(windprofile,windmap))
windzone.to_csv('Processed Data/2022/WindZone_C.csv')

# Solar processing

In [42]:
genlist = pd.read_csv('Public Data/GeneratorList.csv',header=2, encoding='ISO-8859-1', usecols=['Name','SubType','Area Name','InitialDispatch(MW)','Commission Date','DevStatus'])
hourlygenlist = pd.read_csv('Public Data/HourlyResource_General.csv',header=2, encoding='ISO-8859-1')

In [43]:
genlist['Zone'] = genlist['Area Name'].map(region.set_index('Area Name')['Zone'])
genlist = genlist.dropna(subset=['Zone'])
solarlist = hourlygenlist[hourlygenlist['Type'] == 'Solar']
solarlist = pd.merge(solarlist, genlist, left_on='GeneratorName', right_on='Name', how='inner')

In [44]:
columns_to_drop = ['GeneratorKey','Type','CommitmentShapeID','CommitmentMultiplier','DispatchShapeID','DispatchShapeName',
                  'DispatchMultiplier','CommitmentVariation(%)','CommitmentPdf','DispatchVariation(%)','DispatchPdf',
                   'Spillage','IfEnableMultiBlock','ModelType','IfUseInvertLoadingRatio','InvertLoadingRatio','TargetCapacityFactor','Name']
solarlist = solarlist.drop(columns=columns_to_drop)

In [45]:
# solarlist = solarlist[solarlist['Commission Date'] <= '#2023-01-01#']
solarlist = solarlist[solarlist['DevStatus'] == 'Existing']

In [46]:
sum(solarlist[(solarlist['Zone']==1) | (solarlist['Zone']==2)]['Capacity(MW)'])

14141.55

In [47]:
solarlist.to_csv('Processed Data/2022/Solar_C.csv')

# Solar profile


In [48]:
solarshapelist = pd.read_csv('Processed Data/2022/Solar_C.csv', usecols=['GeneratorName','CommitmentShapeName','Capacity(MW)'])
solarcruve1 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Solar1.csv',header=0, skiprows=list(range(8786, 8796)))
solarcruve2 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Solar2.csv',header=0, skiprows=list(range(8786, 8796)))
solarcruve3 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_BTM Solar_DPV.csv',header=0, skiprows=list(range(8786, 8796)))

C:\Users\wenmi\AppData\Local\Temp\ipykernel_10824\1698217285.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  solarcruve1 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Solar1.csv',header=0, skiprows=list(range(8786, 8796)))
C:\Users\wenmi\AppData\Local\Temp\ipykernel_10824\1698217285.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  solarcruve2 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Solar2.csv',header=0, skiprows=list(range(8786, 8796)))
C:\Users\wenmi\AppData\Local\Temp\ipykernel_10824\1698217285.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  solarcruve3 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_BTM Solar_DPV.csv',header=0, skiprows=list(range(8786, 8796)))


In [49]:
solarcruve1 = solarcruve1.drop(index=0)
solarcruve2 = solarcruve2.drop(index=0)
solarcruve3 = solarcruve3.drop(index=0)

In [50]:
solarcruve1.columns = [re.sub(r'\.DAT: 2032$|\.dat: 2032$', '', col_name) for col_name in solarcruve1.columns]
solarcruve2.columns = [re.sub(r'\.DAT: 2032$|\.dat: 2032$', '', col_name) for col_name in solarcruve2.columns]
solarcruve3.columns = [re.sub(r'\.DAT: 2032$|\.dat: 2032$', '', col_name) for col_name in solarcruve3.columns]


solarcruve = pd.merge(solarcruve1, solarcruve2, on='Index')
solarcruve = pd.merge(solarcruve, solarcruve3, on='Index')



In [51]:
solarprofile = pd.DataFrame()
for index, row in solarshapelist.iterrows():
    generator_name = row['GeneratorName']
    solar_profile_name = row['CommitmentShapeName']
    capacity = row['Capacity(MW)']
    solar_profile = solarcruve[solar_profile_name]
    adjusted_profile = solar_profile * capacity
    solarprofile[generator_name] = adjusted_profile

C:\Users\wenmi\AppData\Local\Temp\ipykernel_10824\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  solarprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_10824\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  solarprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_10824\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

In [52]:
solarprofile.to_csv('Processed Data/2022/SolarProfile_C.csv')

# Solar  map

In [53]:
solarzone = pd.read_csv('Processed Data/2022/Solar_C.csv', usecols=['GeneratorName','Zone'])
solarmap = pd.get_dummies(solarzone["Zone"], dtype=int)
solarmap.to_csv('Processed Data/2022/SolarMap_C.csv')

In [54]:
solarprofile = pd.read_csv('Processed Data/2022/SolarProfile_C.csv').iloc[:,1:].values
solarmap = pd.read_csv('Processed Data/2022/SolarMap_C.csv').iloc[:,1:].values

solarzone = pd.DataFrame(np.matmul(solarprofile,solarmap))
solarzone.to_csv('Processed Data/2022/SolarZone_C.csv')

# Hydro  processing

In [55]:
genlist = pd.read_csv('Public Data/GeneratorList.csv',header=2, encoding='ISO-8859-1', usecols=['Name','SubType','Area Name','MinCap(MW)','MaxCap(MW)','InitialDispatch(MW)','DevStatus'])
genlist['Zone'] = genlist['Area Name'].map(region.set_index('Area Name')['Zone'])
genlist = genlist.dropna(subset=['Zone'])
hydrolist = genlist.loc[genlist['SubType'].isin(['Hydro', 'HydroRPS'])]

In [56]:
hydrolist = hydrolist[hydrolist['DevStatus'] == 'Existing']

In [57]:
hydrolist.to_csv('Processed Data/2022/Hydro_C.csv')

In [58]:
hydrocruve = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Hydro by Area.csv',header=0, skiprows=list(range(8786, 8796)))
hydrocruve = hydrocruve.drop(index=0)
hydrocruve.columns = [re.sub('HY_(.*?)_2018.dat: 2032', r'\1', col_name) for col_name in hydrocruve.columns]


In [59]:
hydroprofile = pd.DataFrame()
for index, row in hydrolist.iterrows():
    generator_name = row['Name']
    hydro_profile_name = row['Area Name']
    capacity = row['MaxCap(MW)']
    if hydro_profile_name == 'SRP':
        hydro_profile = hydrocruve['PNM']
    elif hydro_profile_name in {'CISC','CISD'}:
        hydro_profile = hydrocruve['CISC_CISD']
    elif hydro_profile_name in {'CIPV','CIPB'}:
        hydro_profile = hydrocruve['CIPV_CIPB']
    elif hydro_profile_name in {'CHPD','DOPD'}:
        hydro_profile = hydrocruve['GCPD']
    elif hydro_profile_name in {'PSCO','IPMV','IPTV','IPFE'}:
        hydro_profile = hydrocruve['PSCO_IPMV_IPTV_IPFE']
    elif hydro_profile_name == 'WAUW':
        hydro_profile = hydrocruve['WACM']
    elif hydro_profile_name in {'WALC','SPPC'}:
        hydro_profile = hydrocruve['PAUT']
    elif hydro_profile_name == 'TH_Mead':
        hydro_profile = hydrocruve['NEVP']
    else: 
        hydro_profile = hydrocruve[hydro_profile_name]
    adjusted_profile = hydro_profile * capacity
    hydroprofile[generator_name] = adjusted_profile

C:\Users\wenmi\AppData\Local\Temp\ipykernel_10824\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hydroprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_10824\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hydroprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_10824\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

In [60]:
hydroprofile.to_csv('Processed Data/2022/HydroProfile_C.csv')

# Hydro  map


In [61]:
hydrozone = pd.read_csv('Processed Data/2022/Hydro_C.csv', usecols=['Name','Zone'])
hydromap = pd.get_dummies(hydrozone["Zone"], dtype=int)
hydromap.to_csv('Processed Data/2022/HydroMap_C.csv')

# Hydro Zone

In [62]:
hydrozone = pd.DataFrame(np.matmul(hydroprofile.values,hydromap.values))
hydrozone.to_csv('Processed Data/2022/HydroZone_C.csv')

In [68]:
# Ensure no negative values
hydrolist[['MinCap(MW)', 'MaxCap(MW)']] = hydrolist[['MinCap(MW)', 'MaxCap(MW)']].clip(lower=0)

# Sum capacities by zone
zone_sums = hydrolist.groupby('Zone')[['MinCap(MW)', 'MaxCap(MW)']].sum()

# Ensure all zones 1 to 7 are present
for zone in range(1, 8):
    if zone not in zone_sums.index:
        # Add missing zones with 0 values
        zone_sums.loc[zone] = [0, 0]

# Sort the index to ensure the zones are in order
zone_sums.sort_index(inplace=True)

zone_sums.to_csv('Processed Data/2022/HydroZoneCap_C.csv')

# Storage  processing

In [64]:
genlist = pd.read_csv('Public Data/GeneratorList.csv',header=2, encoding='ISO-8859-1', usecols=['Name','SubType','Area Name','MinCap(MW)','MaxCap(MW)','InitialDispatch(MW)','DevStatus'])
genlist['Zone'] = genlist['Area Name'].map(region.set_index('Area Name')['Zone'])
genlist = genlist.dropna(subset=['Zone'])

In [65]:
storagelist = genlist.loc[genlist['SubType'].isin(['PS-Hydro', 'PS-HydroRPS','Battery Storage'])]
storagelist['Efficiency'] = np.where(storagelist['SubType'].isin(['PS-Hydro', 'PS-HydroRPS']), 0.80, 
                            np.where(storagelist['SubType'] == 'Battery Storage', 0.90, np.nan))
storagelist['Duration(h)'] = np.where(storagelist['SubType'].isin(['PS-Hydro', 'PS-HydroRPS']), 12, 
                            np.where(storagelist['SubType'] == 'Battery Storage', 4, np.nan))
condition = storagelist['SubType'] == 'Battery Storage'
storagelist.loc[condition, 'MaxCap(MW)'] = storagelist.loc[condition, 'MaxCap(MW)'] * 2.5
storagelist.loc[condition, 'MinCap(MW)'] = storagelist.loc[condition, 'MinCap(MW)'] * 2.5
storagelist.loc[storagelist['MinCap(MW)'] >= 0, 'MinCap(MW)'] = -storagelist.loc[storagelist['MinCap(MW)'] >= 0, 'MaxCap(MW)']
storagelist['MaxCap(MWh)'] = storagelist['MaxCap(MW)'] * storagelist['Duration(h)']
storagelist['Strategic'] = 0

C:\Users\wenmi\AppData\Local\Temp\ipykernel_10824\4241346663.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  storagelist['Efficiency'] = np.where(storagelist['SubType'].isin(['PS-Hydro', 'PS-HydroRPS']), 0.80,
C:\Users\wenmi\AppData\Local\Temp\ipykernel_10824\4241346663.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  storagelist['Duration(h)'] = np.where(storagelist['SubType'].isin(['PS-Hydro', 'PS-HydroRPS']), 12,
C:\Users\wenmi\AppData\Local\Temp\ipykernel_10824\4241346663.py:10: SettingWithCopyWa

In [67]:
storagelist = storagelist[storagelist['DevStatus'] == 'Existing']

In [69]:
storagelist.to_csv('Processed Data/2022/Storage_C_4hr_5GW_Strategic.csv')

# Storage  map


In [71]:
storagezone = pd.read_csv('Processed Data/2022/Storage_C_4hr_5GW_Strategic.csv', usecols=['Name','Zone'])
storagemap = pd.get_dummies(storagezone["Zone"], dtype=int)
storagemap.to_csv('Processed Data/2022/StorageMap_C.csv')

# Storage  bid


In [72]:
DADBidsQ1 = [280,280,280,280,280,280,265,250,240,245,250,255,260,260,260,260,260,270,240,210,210,210,240,270]
DADBidsQ2 = [260,250,250,250,250,245,240,230,220,220,220,220,220,220,220,220,220,220,220,205,190,215,230,240]
DADBidsQ3 = [250,250,250,250,250,250,250,250,250,250,250,250,250,250,250,240,240,220,205,190,200,225,250,240]
DADBidsQ4 = [250,250,250,250,250,250,250,250,250,250,250,250,250,250,250,250,250,240,225,210,220,235,250,250]

DACBidsQ1 = [-70,-65,-60,-55,-50,-57,-64,-70,-50,0,15,30,40,30,15,0,-20,-40,-60,-70,-70,-70,-50,-60]
DACBidsQ2 = [-50,-50,-50,-50,-50,-50,-50,-50,-40,0,10,20,30,25,20,10,0,-20,-40,-50,-50,-50,-50,-50]
DACBidsQ3 = [-20,-20,-20,-20,-20,-20,-20,-20,-20,0,20,40,40,40,40,40,40,20,0,-20,-20,-20,-20,-20]
DACBidsQ4 = [0,0,0,0,0,0,0,0,0,20,40,50,50,50,40,25,10,0,0,0,0,0,0,0]

RTDBidsQ1 = [150,150,150,145,140,135,130,125,140,150,150,150,150,150,147,143,140,110,80,50,70,90,110,130]
RTDBidsQ2 = [160,170,180,180,180,170,170,185,200,185,170,155,140,140,140,140,140,140,115,90,75,100,125,150]
RTDBidsQ3 = [190,200,200,200,200,190,180,200,210,205,200,200,200,200,195,195,190,170,140,110,100,125,150,175]
RTDBidsQ4 = [200,250,250,250,250,250,235,220,230,235,240,230,220,220,220,215,210,190,170,150,160,185,210,230]

RTCBidsQ1 = [-60,-50,-50,-50,-50,-50,-50,-50,-50,0,10,20,30,20,10,0,-50,-50,-50,-50,-50,-50,-50,-50]
RTCBidsQ2 = [-40,-40,-40,-40,-50,-50,-50,-50,0,30,40,50,40,35,30,20,10,-20,-50,-40,-40,-40,-40,-40]
RTCBidsQ3 = [-40,-25,-21,-17,-15,-20,-25,-30,-15,50,80,85,80,70,64,58,50,25,0,-5,-10,-15,-20,-25]
RTCBidsQ4 = [0,0,5,10,5,0,0,0,10,40,70,100,125,125,125,75,30,15,0,0,0,0,0,0]

In [73]:
DADBids = DADBidsQ1 * 90 + DADBidsQ2 * 91 + DADBidsQ3 * 92 + DADBidsQ4 * 92
DACBids = DACBidsQ1 * 90 + DACBidsQ2 * 91 + DACBidsQ3 * 92 + DACBidsQ4 * 92
RTDBids = RTDBidsQ1 * 90 + RTDBidsQ2 * 91 + RTDBidsQ3 * 92 + RTDBidsQ4 * 92
RTCBids = RTCBidsQ1 * 90 + RTCBidsQ2 * 91 + RTCBidsQ3 * 92 + RTCBidsQ4 * 92

In [ ]:
ESDABids = pd.DataFrame({'Discharge': DADBids, 'Charge': DACBids})
ESRTBids = pd.DataFrame({'Discharge': RTDBids, 'Charge': RTCBids})
ESRTBids_repeated = ESRTBids.loc[ESRTBids.index.repeat(12)].reset_index(drop=True)

# Save to CSV
ESDABids.to_csv('Processed Data/2022/StorageDABids.csv',index=False)
ESRTBids_repeated.to_csv('Processed Data/2022/StorageRTBids.csv',index=False)